In [1]:
import numpy as np
import pandas as pd

import delicatessen
from delicatessen import GMMEstimator
from delicatessen.estimating_equations import ee_regression, ee_rogan_gladen
from delicatessen.utilities import inverse_logit

print("Versions")
print("NumPy:        ", np.__version__)
print("pandas:       ", pd.__version__)
print("Delicatessen: ", delicatessen.__version__)

Versions
NumPy:         1.25.2
pandas:        1.4.1
Delicatessen:  3.0


In [2]:
n = 500
d = pd.DataFrame()
d['Z1'] = np.random.binomial(n=1, p=0.5, size=n)
d['Z2'] = np.random.binomial(n=1, p=0.5, size=n)
d['U'] = np.random.normal(size=n)
d['A'] = np.random.binomial(n=1, p=inverse_logit(d['U'] + d['Z1'] - d['Z2']), size=n)
d['Y'] = 2*d['A'] - d['U'] + np.random.normal(size=n)

In [3]:
d.describe()

,Z1,Z2,U,A,Y
count,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.500000,0.540000,-0.018708,0.510000,1.125005
std,0.500501,0.498897,1.051151,0.500401,1.555370
min,0.000000,0.000000,-2.956990,0.000000,-3.477546
25%,0.000000,0.000000,-0.708308,0.000000,0.107649
50%,0.500000,1.000000,-0.019057,1.000000,1.140912
75%,1.000000,1.000000,0.726228,1.000000,2.205340
max,1.000000,1.000000,2.788880,1.000000,5.323891


In [4]:
z1 = np.asarray(d['Z1'])
a = np.asarray(d['A'])
y = np.asarray(d['Y'])


def psi(theta):
    return (y*z1 - y*(1-z1)) - theta*(a*z1 - a*(1-z1))

estr = GMMEstimator(psi, init=[0.,])
estr.estimate()

Theta solution [1.28566756]
Theta solution [1.28566756]


In [5]:
z1 = np.asarray(d['Z1'])
z2 = np.asarray(d['Z2'])
a = np.asarray(d['A'])
y = np.asarray(d['Y'])


def psi(theta):
    ee_z1 = (y*z1 - y*(1-z1)) - theta*(a*z1 - a*(1-z1))
    ee_z2 = (y*z2 - y*(1-z2)) - theta*(a*z2 - a*(1-z2))
    return np.vstack([ee_z1, ee_z2])

estr = GMMEstimator(psi, init=[0.,])
estr.estimate()

ValueError: The number of initial values should be greater than or equal to the number of rows returned by `stacked_equations` but there are 1 initial values and the `stacked_equations` function returns 2 row(s).